In [ ]:
import hermes
from hermes.instruments.EBSD.some_manufacturer import some_instrument
from hermes.pre_proccessing.EBSD import read_ang_files
###########
#Example: Active learning for phase mapping, grain mapping, etc.
###########
#####
#specific example EBSD for 3D grain mapping
#####
#Define a way to to communicate with instrument
path = "C://some_path"
some_instrument.set_directory(path)
possible_locations = something???

def change_z(depth):
    hermes.instruments.EBSD.mill(depth)

def move_and_measure(locations, count_time= 5):
    #count_time in seconds
    some_instrument.move(locations)
    
    some_instrument.measure(count_time)
    #^maybe .load_ang() should be a part of .measure()
    some_instrument.verify_locations()
    
    #Preprocess data
    real_locations, measurments = some_instrument.load_ang(path)
    
    return real_locations, measurements
    


#Define active learning loop:
active_learning_loops = 10
#initialize the starting locations
locations = some_instrument.initial_locations(method = "random")
possible_locations = some_instrument.set_range()

for i in active_learning_loops:
    #Measure
    un_measured_locations = np.setdiff1d(locations, possible_locations) # Find the locations that haven't been measured. 
    #^needs some sort of tolerance 
    new_locations, new_measurements = move_and_measure(un_measured_locations) 
    locations = np.concatenate((locations, new_locations), axis=0)
    measurements = np.concatenate((measurements, new_measurements), axis=0)
    
    
    #Clustering
    clusters, cluster_uncertainties = hermes.ida.cluster.ContigousCommunityDiscovery.rb_pots(locations, measurements)


    #Train-Predict-Acquire
    model = some_model.train(locations, clusters, cluster_uncertainties)
    predictions, prediction_uncertainties = model.predict(possible_locations)
    
    next_locations = some_acquistion_method(predictions, prediction_uncertainties)
    
    #Archive data
    some_archive_method(i,
                        locations, measurements, 
                        clusters, cluster_uncertainties,
                        model, 
                        predictions, prediction_uncertainties
                        next_locations)
    
    #Add next location to the list of locations to measure
    locations = np.concatenate((locations, next_locations), axis = 0)